In [11]:
x = [[0., 4., 2.], [1., 3., 5., -1.0]]
y = [2*x[1], 4*x[2]]

2-element Vector{Vector{Float64}}:
 [0.0, 8.0, 4.0]
 [4.0, 12.0, 20.0, -4.0]

In [5]:


function sort_spectra(x::Vector{Float64}, y::Vector{Float64})
    p = sortperm(x)
	return x[p], y[p]
end
function sort_spectra(x::Vector{<:Vector}, y::Vector{<:Vector})
    # method for collection of x and y inputs
    return [sort_spectra(x, y) for (x, y) in zip(x, y)]
end

sort_spectra (generic function with 2 methods)

In [12]:
sort_spectra(x, y)

2-element Vector{Tuple{Vector{Float64}, Vector{Float64}}}:
 ([0.0, 2.0, 4.0], [0.0, 4.0, 8.0])
 ([-1.0, 1.0, 3.0, 5.0], [-4.0, 4.0, 12.0, 20.0])

In [20]:
function flipsp(spectra::Matrix{Float64})
    x = spectra[:,1]
    p = sortperm(x)
    return spectra[p,:]
end
function flipsp(spectra::Vector{<:Matrix})
    return [flipsp(sp) for sp in spectra]
end

flipsp (generic function with 4 methods)

In [21]:
x = [0., 2.,5.,-1]
x2 = [0., 2.,5.,-1, 3., -10.]
y = 2*x
y2 = 2*x2

sp1 = [x y]

sp2 = [x2 y2]

sps = [sp1, sp2]

2-element Vector{Matrix{Float64}}:
 [0.0 0.0; 2.0 4.0; 5.0 10.0; -1.0 -2.0]
 [0.0 0.0; 2.0 4.0; … ; 3.0 6.0; -10.0 -20.0]

In [1]:
using DataInterpolations

In [24]:
function resample(x::Vector{Float64},y::Vector{Float64},x_new::Vector{Float64}; method::String = "AkimaInterpolation")
    # this is the parent method: treat x-y Vectors
    if length(x) != length(y)
        throw(ArgumentError("x and y must have the same length"))
    end
    p = sortperm(x)# we automatically sort the data
    interp = getfield(DataInterpolations, Symbol(method))(y[p], x[p]; extrapolation_right=ExtrapolationType.Extension, extrapolation_left=ExtrapolationType.Extension)
    
    return interp(x_new)
end
function resample(x::Vector{Float64},y::Matrix{Float64},x_new::Vector{Float64}; method::String = "AkimaInterpolation")
    # this method treats an array of y signals with a common X
    out = ones((size(x_new,1), size(y,2)))
    for i = 1:size(y,2)
        out[:,i] = resample(x, vec(y[:,i]), x_new; method=method)
    end
    return out
end
function resample(multiple_spectra::Vector{<:Matrix{Float64}},x_new::Vector{Float64}; method::String = "AkimaInterpolation")
    # this method treats a list of input spectra
    out = ones((size(x_new,1), size(multiple_spectra,1)))
    for (index,i) in enumerate(multiple_spectra)
        out[:, index] = resample(i[:,1], i[:,2], x_new; method=method)
    end
    return out
end

resample (generic function with 3 methods)

In [26]:
x1 = [0., 2., 4.]
y1 = 2.0*x1
x_new = [1., 3.]

# method 1
y_new = resample(x1, y1, x_new)
@test y_new == 2.0*x_new

x2 = [8., 4., 1., 10.]
y2 = [2.0*x2 x2]

# method 2
y_new = resample(x2, y2, x_new)
@test y_new == [2.0*x_new x_new]

# method 3
y_new = resample([[x1 y1], [x1 y1]], x_new)
@test y_new == [2.0*x_new 2.0*x_new]

Test Passed

In [27]:
x1 = [0., 2., 4.]
y1 = 2.0*x1
x_new = [1., 3.]

# method 1
y_new = resample(x1, y1, x_new)

2-element Vector{Float64}:
 2.0
 6.0

In [28]:
y1

3-element Vector{Float64}:
 0.0
 4.0
 8.0

In [32]:
using Spectra

LoadError: failed to find source of parent package: "LoopVectorization"

In [7]:
# signal creation
x = collect(0.:0.8:10.)
y, ys = gaussiennes([10.,5.], [4.,6.], [1.1,0.8], x)

# the new x axis
x_new = collect(0.:0.05:10.)

# resampling y as a vector
y2 = resample(x, vec(y), x_new)

# resampling the ys array of the two peaks
y3 = resample(x, ys, x_new)

y_true, ys_true = gaussiennes([10.,5.], [4.,6.], [1.1,0.8], x_new)

# plotting
p1 = scatter(x,y, label="Original data")
plot!(x_new, y_true, label="True y")
plot!(x_new,y2, label="Resampled y")
display(p1)

p2 = scatter(x, ys, label="Original peak data")
plot!(x_new, ys_true, label="True peaks data")
plot!(x_new, y3, label="Resampled peaks")
display(p2)

# resampling a collection of spectra
x = collect(0.:0.8:10.)
y, ys = gaussiennes([10.,5.], [4.,6.], [1.1,0.8], x)
x2 = collect(0.:0.8:10.)
y2, ys2 = gaussiennes([10.,5.], [4.,6.], [1.1,0.8], x2)
x3 = collect(0.:0.8:10.)
y3, ys3 = gaussiennes([10.,5.], [4.,6.], [1.1,0.8], x3)

spectra_ = [[x y], [x2 y2], [x3 y3]]
x_new = collect(0.:0.05:10.)
spectra_resampled = resample(spectra_, x_new)
p3 = scatter(x, [y, y2, y3], label="Original data")
plot!(x_new, spectra_resampled, label="Resampled data")
display(p3)

2×2 Matrix{Float64}:
 2.0  1.0
 6.0  3.0

In [68]:
size(y,2)

2